# Sam I Am

## Description

Imagine you have a child who wants to know which character is talking in a story. What you can do is running the dialog through a model and see what it thinks. 

At some level this is sentiment analysis. Instead of pick a value you are pick whom is talking.

The first step is generating the story, and this one is shamelessly lifted since it has a small vocab

In [7]:
records = []
def storyAppend(line,who):
    records.append([line.split(" "),who])

storyAppend("i am sam",1)
records

[[['i', 'am', 'sam'], 1]]

In [8]:
storyAppend("sam I am",1)
storyAppend("I do not like that sam I am",0)
storyAppend("do you like green eggs and ham",1)
storyAppend("I do not like green eggs and ham",0)
storyAppend("would you eat them here or there",1)
storyAppend("I would not eath them anywhere",0)
storyAppend("would you eat them here or there",1)
storyAppend("I would not eat them anywhere",0)
storyAppend("would you eat them with a fox",1)
storyAppend("I would not eat them with a fox",0)
storyAppend("could you eat them in a box",1)
storyAppend("I could not eat them in a box I would not eat them with a fox I do not like green eggs and ham sam I am",0)
storyAppend("would you eat them with a goat",1)
storyAppend("I would not eat them with a goat",0)
storyAppend("could you eat them in a boat",1)
storyAppend("I could not eat them in a boat I would not eat them with a goat",0)
storyAppend("would you eat them in the rain",1)
storyAppend("I would not eat them in the rain",0)
storyAppend("could you eat them on a train",1)
storyAppend("I could not eat them in the rain I would not eat them on a train I do not like green eggs and ham sam I am",0)


The next step is create an index for each of the words so we can look them up. Here we sort alphabetically since we are goint to use the vlookup function, but we really could have assigned anything.

In [9]:
wordsToIndex = {}
indexToWords = {}

nextword = 1

for rec in records:
    text = rec[0]
    for word in text:
        wordsToIndex[word] = 1
for word in sorted(wordsToIndex.keys()):
    wordsToIndex[word] = nextword
    indexToWords[nextword] = word
    nextword += 1
    
for i in range(1, nextword -1):
    word = indexToWords[i]
    iWord = wordsToIndex[word]
    print str(i) + " \t" + word +"\t" + str(iWord)

1 	I	1
2 	a	2
3 	am	3
4 	and	4
5 	anywhere	5
6 	boat	6
7 	box	7
8 	could	8
9 	do	9
10 	eat	10
11 	eath	11
12 	eggs	12
13 	fox	13
14 	goat	14
15 	green	15
16 	ham	16
17 	here	17
18 	i	18
19 	in	19
20 	like	20
21 	not	21
22 	on	22
23 	or	23
24 	rain	24
25 	sam	25
26 	that	26
27 	the	27
28 	them	28
29 	there	29
30 	train	30
31 	with	31
32 	would	32


The next step is to create sentences of the same size. This is going to have two parts.
-  We want to find the average length and pad or the other sentences
-  we want to convert the sentences to there corresponding indexes so we can do lookup

In [10]:
avgcount = 0
for rec in records:
    text = rec[0]
    avgcount += len(text)
seq_len = int(avgcount/len(records))
seq_len

9

We can now create mapping that go back and forth between the sentences

In [11]:
def wordListToIndexList(wordArray):
    nums = []
    for i in range(0,seq_len - 1):
        num = 0
        # do we need to pad
        if i < len(wordArray):
            num = wordsToIndex[wordArray[i]]
        nums.append(num)
    return nums

def indexListToString(indexArray):
    sentence = indexToWords[indexArray[0]]
    for i in range(1,seq_len - 1):
        wordIndex = indexArray[i]
        if wordIndex in indexToWords:
            sentence = sentence + " " + indexToWords[wordIndex]

    return sentence

testSentence = wordListToIndexList(["I", "like", "boat"])
print(testSentence)
print(indexListToString(testSentence))

[1, 20, 6, 0, 0, 0, 0, 0]
I like boat


In [12]:
trimmedlines = []

  
for rec in records:
    text = rec[0]
    trimmedlines.append([wordListToIndexList(text),rec[1]])

    #now print it
    
for rec in trimmedlines:
    nums = rec[0]
    base = str(nums[0])
    for i in range(1,seq_len - 1):
        base = base + "\t" + str(nums[i])
    print base +"\t\t" + str(rec[1])

18	3	25	0	0	0	0	0		1
25	1	3	0	0	0	0	0		1
1	9	21	20	26	25	1	3		0
9	33	20	15	12	4	16	0		1
1	9	21	20	15	12	4	16		0
32	33	10	28	17	23	29	0		1
1	32	21	11	28	5	0	0		0
32	33	10	28	17	23	29	0		1
1	32	21	10	28	5	0	0		0
32	33	10	28	31	2	13	0		1
1	32	21	10	28	31	2	13		0
8	33	10	28	19	2	7	0		1
1	8	21	10	28	19	2	7		0
32	33	10	28	31	2	14	0		1
1	32	21	10	28	31	2	14		0
8	33	10	28	19	2	6	0		1
1	8	21	10	28	19	2	6		0
32	33	10	28	19	27	24	0		1
1	32	21	10	28	19	27	24		0
8	33	10	28	22	2	30	0		1
1	8	21	10	28	19	27	24		0


We can now copy this to our excel spreadsheet and run from there but first we can check to see if it makes sense

In [13]:
for rec in trimmedlines:
    nums = rec[0]

    print(indexListToString(nums))


i am sam
sam I am
I do not like that sam I am
do you like green eggs and ham
I do not like green eggs and ham
would you eat them here or there
I would not eath them anywhere
would you eat them here or there
I would not eat them anywhere
would you eat them with a fox
I would not eat them with a fox
could you eat them in a box
I could not eat them in a box
would you eat them with a goat
I would not eat them with a goat
could you eat them in a boat
I could not eat them in a boat
would you eat them in the rain
I would not eat them in the rain
could you eat them on a train
I could not eat them in the rain


## step 2 using keras

In [14]:
from theano.sandbox import cuda
cuda.use('gpu')

Mapped name None to device cuda: GeForce GTX 1080
Using cuDNN version 5105 on context None
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)


In [15]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [16]:
#lets check some of the parameters he used.
vocab_size = len(trimmedlines)
numFilters = 2
denseSize = 5

print(vocab_size, seq_len)

21 9


In [17]:
model = Sequential([
    keras.layers.embeddings.Embedding(vocab_size, numFilters, input_length=seq_len),
    Flatten(),
    Dense(denseSize, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [18]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 9, 2)          42          embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 18)            0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 5)             95          flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 5)             0           dense_1[0][0]                    
___________________________________________________________________________________________

In [19]:
trn = []
labels_train = []
for rec in trimmedlines:
    trn.append(rec[0])
    labels_train.append(rec[1])

In [ ]:
This is where I break. my model isn't in t'

In [20]:
model.fit(trn, labels_train, validation_data=(trn, labels_train), nb_epoch=2, batch_size=8)

Exception: Error when checking model input: expected embedding_input_1 to have shape (None, 9) but got array with shape (21, 8)